In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
from datasets import TxtDataset

In [2]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)

    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

In [3]:
class WordAVGModel(pl.LightningModule):
    def __init__(self, conf):
        super().__init__()
        self.conf = conf
        self.embedding = nn.Embedding(conf.vocab_size, conf.embed_size, padding_idx=0)
        self.fc = nn.Linear(conf.embed_size, conf.num_classes)
        self.dropout = nn.Dropout(conf.dropout)
    
    def forward(self, x):
        embedded = self.dropout(self.embedding(x))  # [B, seq, embed_size]
        pooled = F.avg_pool2d(embedded, (embedded.size(1), 1)).squeeze(1)
        return self.fc(pooled)
    
    def prepare_data(self):
        self.train_set = TxtDataset(f"{self.conf.data_path}/cnews.train.txt", maxlen=512)
        self.val_set = TxtDataset(f"{self.conf.data_path}/cnews.val.txt", maxlen=512)
        self.test_set = TxtDataset(f"{self.conf.data_path}/cnews.test.txt", maxlen=512)
    
    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=32, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=32, shuffle=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=32, shuffle=True)
    
    def _process_one_batch(self, batch, flag='train'):
        x, y = batch
        y_hat = self(x)
        loss_func = nn.CrossEntropyLoss()
        loss = loss_func(y_hat.view(-1, self.conf.num_classes), y.view(-1))
        self.log(f'{flag}_loss', loss)

        _, y_pred = torch.max(y_hat.view(-1, self.conf.num_classes), dim=-1)
        acc = accuracy_score(y_pred.cpu(), y.cpu())
        acc = torch.tensor(acc)
        self.log(f'{flag}_accuracy', acc)

        return loss 

    def training_step(self, batch, batch_nb):
        loss = self._process_one_batch(batch, flag='train')
        return loss
    
    def validation_step(self, batch, batch_nb):
        return self._process_one_batch(batch, flag='val')
    
    def test_step(self, batch, batch_nb):
        return self._process_one_batch(batch, flag='test')

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.conf.lr)
        return optimizer

In [4]:
def main(conf):
    model = WordAVGModel(conf)
    tb_logger = pl_loggers.TensorBoardLogger('logs/')
    ckpt = ModelCheckpoint(
        filepath=conf.model_name,
        verbose=False,
        monitor='val_loss',
        mode='min'
    )
    trainer = pl.Trainer(
        max_epochs=10,
        logger=tb_logger,
        checkpoint_callback=ckpt,
    )

    trainer.fit(model)
    trainer.test(ckpt_path=trainer.checkpoint_callback.best_model_path)

In [5]:
conf = Config(
    model_name='word_avg',
    data_path=Path(r'/Users/liuzhi/datasets/cnews'),
    vocab_size=50000,
    embed_size=300,
    num_classes=10,
    lr=3e-3,
    dropout=0.2
)

In [6]:
main(conf)

it/s, loss=0.012, v_num=0]
Epoch 9: 100%|██████████| 1720/1720 [04:49<00:00,  5.94it/s, loss=0.001, v_num=0]
35553it [01:45, 235.79it/s]error line : 蚂蚁的连击，这个么，单纯懒得去验证，而且我感觉近战用这个技能才更需要测试。圣堂的贯穿式攻击、tb龙骑变身后的远程溅射，这个，理论上应该是不算在以上四个等级的攻击特效里面的，所以属于完全不受影响的技能。依然那句话，ak不到无法测试……攻击特效之间的相互影响（未测试，仅提出想法）： 不知道有多少人看过以前那个法球效果表，dota正常版里法球绝大部分是无法叠加的，优先级一般是自动施放法球>吸血面具>其他法球，其他法球中的优先级顺序为物品栏中靠前的法球优先，最后获得的法球优先。唯一一个例外是远程英雄的冰眼，以及龙骑三段变身的霜冻攻击，这两个是可以和吸血面具叠加的。除此之外，冰眼还有一个十分强大的功能，就是强化法球。具体内容记不太清楚了，只晓得冰眼和散夜或者雷锤同时带身上，可以提高触发概率。同时冰眼的效果会受到英雄本身近战/远程状态的影响，tb巨魔龙骑这种会变来变去的英雄在某个状态捡起冰眼之后变成另一个状态，冰眼的状态会保持原本的近战或者远程状态。imba版本里虽然法球之间的冲突已经基本全部消失了，可是对于冰眼这么奇妙欢乐的玩意儿，说不准还真的挺有内容的。更多衍生的想法：imba里面的攻击特效已经非常多了，我发现米米亚也开始逐步增加施法特效了。先是火女宙斯，然后卡拉波和60智力法杖，现在蓝胖被解禁，冰甲加入了新的im特效。我建议可以加入更多施法特效的物品，目前只有冰甲和卡拉波，而且效果也不怎么看得出来，永恒长夜法杖又是个基本没人出的货。法系装备略贵，也略少。另一方面，我个人玩imba纯粹为了娱乐，所以也不在乎平衡不平衡的问题，一切归结于脸。所以我觉得imba如果依然是想要往娱乐的方向发展的话，可以将随机性的成分继续放大。幽鬼折射就是个很好的例子，从百分比的硬免伤，变成了几率性的全面伤，人马一个双刃剑弹死自己的现象屡见不鲜。所以人马刚毅不屈和死灵龙守墓人斗篷之类的技能，我觉得也可以往几率性全免伤上面靠。而同时，很多ak技能废物自己就基本成渣的英雄，可以考虑加强一下。比如谜团之流。而那些